In [1]:
!python train.py --model bert2 --drop_out=0.35 --bert_layers=1 --max_seq_len=20 --hidden_dim=100 --n_epochs=5 --n_heads=8 --file_name=elo.csv --lr=0.0003


-------------------- conti_feats -------------------
['tag_mean', 'tag_std', 'ass0_mean', 'ass0_std', 'ass1_mean', 'ass1_std', 'ass2_mean', 'ass2_std', 'test0_mean', 'test0_std', 'test1_mean', 'test1_std', 'elapsed']
-------------------- cate_feats --------------------
['KnowledgeTag', 'month', 'hour', 'week', 'assessmentItemID0', 'assessmentItemID1', 'assessmentItemID2', 'testId0', 'testId1', 'elapsed_cate', 'as0_as1', 'as0_as2', 'as1_as2', 'assessmentItemID', 'week_hour']
----------------------------------------------------
random data seed : 1824
wandb: Currently logged in as: kyjun (recsys_lvl2). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.13.5
wandb: Run data is saved locally in /opt/ml/input/code/dkt/wandb/run-20221202_072343-2p9vxobm
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run fast-silence-2094
wandb: ⭐️ View project at https://wandb.ai/recsys_lvl2/dkt
wandb: 🚀 View run at https://wandb.ai/recsys_lvl2/dkt/runs/2p9

In [1]:
!python train.py --n_epochs=20 --bert_layers=1 --drop_out=0.34845223154374694 --file_name=elo.csv --hidden_dim=91 --hidden_size=767 --lr=0.0005082132287941777 --max_seq_len=141 --model=bert2 --n_heads=27


-------------------- conti_feats -------------------
['tag_mean', 'tag_std', 'ass0_mean', 'ass0_std', 'ass1_mean', 'ass1_std', 'ass2_mean', 'ass2_std', 'test0_mean', 'test0_std', 'test1_mean', 'test1_std', 'elapsed']
-------------------- cate_feats --------------------
['KnowledgeTag', 'month', 'hour', 'week', 'assessmentItemID0', 'assessmentItemID1', 'assessmentItemID2', 'testId0', 'testId1', 'elapsed_cate', 'as0_as1', 'as0_as2', 'as1_as2', 'assessmentItemID', 'week_hour']
----------------------------------------------------
random data seed : 1824
wandb: Currently logged in as: kyjun (recsys_lvl2). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.13.5
wandb: Run data is saved locally in /opt/ml/input/code/dkt/wandb/run-20221202_061943-1583sbwc
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run tough-spaceship-2083
wandb: ⭐️ View project at https://wandb.ai/recsys_lvl2/dkt
wandb: 🚀 View run at https://wandb.ai/recsys_lvl2/dkt/runs/

In [278]:
!wandb agent recsys_lvl2/dkt/zoobasx0

wandb: Starting wandb agent 🕵️
2022-12-02 01:11:05,788 - wandb.wandb_agent - INFO - Running runs: []
2022-12-02 01:11:07,367 - wandb.wandb_agent - INFO - Agent received command: run
2022-12-02 01:11:07,368 - wandb.wandb_agent - INFO - Agent starting run with config:
	bert_layers: 1
	drop_out: 0.462183659258095
	file_name: elo.csv
	hidden_dim: 44
	hidden_size: 660
	lr: 0.00023697342801797333
	max_seq_len: 65
	model: bert2
	n_heads: 24
2022-12-02 01:11:07,370 - wandb.wandb_agent - INFO - About to run command: /usr/bin/env python train.py --bert_layers=1 --drop_out=0.462183659258095 --file_name=elo.csv --hidden_dim=44 --hidden_size=660 --lr=0.00023697342801797333 --max_seq_len=65 --model=bert2 --n_heads=24

-------------------- conti_feats -------------------
['tag_mean', 'tag_std', 'ass0_mean', 'ass0_std', 'ass1_mean', 'ass1_std', 'ass2_mean', 'ass2_std', 'test0_mean', 'test0_std', 'test1_mean', 'test1_std', 'elapsed', 'user_correct_answer', 'user_acc']
-------------------- cate_feats --

In [154]:
import torch
class DKTDataset2(torch.utils.data.Dataset):
    def __init__(self, df, embed):
        self.df = df
        self.seq_len = 2
        self.cate = ['KnowledgeTag', 'month', 'hour', 'week', 'assessmentItemID0', 'assessmentItemID1', 'assessmentItemID2', 'testId0', 'testId1', 'elapsed_cate', 'as0_as1', 'as0_as2', 'as1_as2', 'assessmentItemID', 'week_hour']
        self.conti = ['tag_mean', 'tag_std', 'ass0_mean', 'ass0_std', 'ass1_mean', 'ass1_std', 'ass2_mean', 'ass2_std', 'test0_mean', 'test0_std',
    'test1_mean', 'test1_std', 'elapsed']
        self.embed = embed
    def __getitem__(self, index):
        data = self.df[index*self.seq_len:(index+1)*self.seq_len]
        cate = torch.tensor(data[self.cate].values)
        conti = torch.FloatTensor(data[self.conti].values)
        
        for i, j in enumerate(self.embed):
            cate[:, i] += j
        
        answer = data['answerCode'].values
        
        return cate, conti, answer

    def __len__(self):
        return int(len(self.df) // self.seq_len) + 1

In [143]:
cate = ['KnowledgeTag', 'month', 'hour', 'week', 'assessmentItemID0', 'assessmentItemID1', 'assessmentItemID2', 'testId0', 'testId1', 'elapsed_cate', 'as0_as1', 'as0_as2', 'as1_as2', 'assessmentItemID', 'week_hour']
e = []
trian = train[-10000:]
for i in cate:
    e.append(train[i].nunique())
embed = np.array([0, *np.cumsum(e)[:-1]])
trainset = DKTDataset2(train, embed)
train_loader = torch.utils.data.DataLoader(
    trainset,
    shuffle=True,
    batch_size=128,
    drop_last=True,
    collate_fn=collate,
)

In [202]:
def collate(batch):
    col_n = len(batch[0])
    col_list = [[[],[],[]] for _ in range(col_n)]
    max_seq_len = len(batch[0][-1])
    len(batch[-1][0])
#     for types in batch:
#         for tt in types:
#             for i, col in enumerate(types):
#                 pre_padded = torch.zeros(max_seq_len)
#                 print(pre_padded, len(col))
#                 pre_padded[-len(col) :] = col
#                 col_list[i].append(pre_padded)

#     for i, _ in enumerate(col_list):
#         col_list[i] = torch.stack(col_list[i])

    return tuple(batch[0])
trian = train[-21:-57]
trainset = DKTDataset2(train, embed)
train_loader = torch.utils.data.DataLoader(
    trainset,
    shuffle=True,
    batch_size=2,
    drop_last=True,
    collate_fn=collate,
)

In [203]:
a = 0
for i,j,k in train_loader:

    if torch.max(i) > a:
        a = torch.max(i)
    print(i,j,k)
        
print(a)
#b s f

tensor([[   34,   913,   947,   948,   956,  1053,  1164,  1176,  1273,  1384,
          1914,  2935,  3661,  8025, 14227],
        [   30,   913,   947,   948,   956,  1053,  1165,  1176,  1273,  1384,
          1914,  2936,  3662,  8026, 14227]]) tensor([[ 0.5403,  0.4985,  0.5002,  0.5008,  0.6150,  0.4866,  0.6883,  0.4632,
          0.5002,  0.5008,  0.6150,  0.4866, 69.0000],
        [ 0.5709,  0.4951,  0.5002,  0.5008,  0.6150,  0.4866,  0.6635,  0.4727,
          0.5002,  0.5008,  0.6150,  0.4866, 94.0000]]) [1 1]
tensor([[  201,   923,   926,   952,   958,  1006,  1165,  1178,  1226,  1383,
          2580,  2956,  3333, 12023, 14168],
        [  201,   923,   926,   952,   958,  1006,  1168,  1178,  1226,  1383,
          2580,  2966,  3335, 12025, 14168]]) tensor([[0.6844, 0.4648, 0.6797, 0.4673, 0.6416, 0.4796, 0.6635, 0.4727, 0.6797,
         0.4673, 0.6416, 0.4796, 2.0000],
        [0.6844, 0.4648, 0.6797, 0.4673, 0.6416, 0.4796, 0.5562, 0.4977, 0.6797,
         0.4673, 0.

KeyboardInterrupt: 

In [260]:
t = pd.read_csv('sun.csv')
a = pd.read_csv('output_0.8346.csv')

b = list(t['prediction'].values) + list(a['prediction'][-24:].values)
a['prediction'] = b
a.to_csv('sun2.csv', index=False)

In [284]:
t = pd.read_csv('sun.csv')
a = pd.read_csv('output_0.8346.csv')
a['prediction'] = t[:744].values
a.to_csv('sun2.csv', index=False)

In [4]:
import pandas as pd
a = pd.read_csv('../../data/infer.csv')
b = pd.concat([a, a])
b

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,assessmentItemID0,assessmentItemID1,assessmentItemID2,testId0,...,ass0_mean,ass0_std,ass1_mean,ass1_std,ass2_mean,ass2_std,test0_mean,test0_std,test1_mean,test1_std
0,3,2641,A050000133,-1,2020-10-26 13:13:57,369,6,131,7,6,...,0.658649,0.474735,0.653103,0.477301,0.452308,0.502379,0.658649,0.474735,0.653103,0.477301
1,4,3264,A070000146,-1,2020-12-27 02:47:54,579,7,153,7,7,...,0.521167,0.500333,0.749312,0.437321,0.452308,0.502379,0.521167,0.500333,0.749312,0.437321
2,13,6174,A070000111,-1,2020-12-27 04:35:09,549,7,144,7,7,...,0.521167,0.500333,0.647933,0.477910,0.452308,0.502379,0.521167,0.500333,0.647933,0.477910
3,17,7409,A090000064,-1,2020-10-30 05:48:37,849,4,116,6,4,...,0.454470,0.498214,0.648398,0.477672,0.556200,0.497726,0.454470,0.498214,0.648398,0.477672
4,26,5608,A060000135,-1,2020-10-23 11:44:18,617,0,134,5,0,...,0.711898,0.453371,0.644013,0.481474,0.514423,0.501678,0.711898,0.453371,0.644013,0.481474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,7395,5941,A040000122,-1,2020-09-08 02:05:20,284,3,175,4,3,...,0.679714,0.467279,0.672852,0.470161,0.598288,0.491629,0.679714,0.467279,0.672852,0.470161
740,7404,6879,A030000111,-1,2020-10-13 09:49:18,673,5,144,4,5,...,0.701733,0.458110,0.647933,0.477910,0.598288,0.491629,0.701733,0.458110,0.647933,0.477910
741,7416,9017,A050000193,-1,2020-10-04 02:44:41,878,6,184,3,6,...,0.658649,0.474735,0.770746,0.422156,0.663468,0.472688,0.658649,0.474735,0.770746,0.422156
742,7417,9017,A050000193,-1,2020-09-06 13:09:15,878,6,184,3,6,...,0.658649,0.474735,0.770746,0.422156,0.663468,0.472688,0.658649,0.474735,0.770746,0.422156


In [5]:
b.to_csv('../../data/infer2.csv', index=False)

In [6]:
c = pd.read_csv('../../data/infer2.csv')
c

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,assessmentItemID0,assessmentItemID1,assessmentItemID2,testId0,...,ass0_mean,ass0_std,ass1_mean,ass1_std,ass2_mean,ass2_std,test0_mean,test0_std,test1_mean,test1_std
0,3,2641,A050000133,-1,2020-10-26 13:13:57,369,6,131,7,6,...,0.658649,0.474735,0.653103,0.477301,0.452308,0.502379,0.658649,0.474735,0.653103,0.477301
1,4,3264,A070000146,-1,2020-12-27 02:47:54,579,7,153,7,7,...,0.521167,0.500333,0.749312,0.437321,0.452308,0.502379,0.521167,0.500333,0.749312,0.437321
2,13,6174,A070000111,-1,2020-12-27 04:35:09,549,7,144,7,7,...,0.521167,0.500333,0.647933,0.477910,0.452308,0.502379,0.521167,0.500333,0.647933,0.477910
3,17,7409,A090000064,-1,2020-10-30 05:48:37,849,4,116,6,4,...,0.454470,0.498214,0.648398,0.477672,0.556200,0.497726,0.454470,0.498214,0.648398,0.477672
4,26,5608,A060000135,-1,2020-10-23 11:44:18,617,0,134,5,0,...,0.711898,0.453371,0.644013,0.481474,0.514423,0.501678,0.711898,0.453371,0.644013,0.481474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483,7395,5941,A040000122,-1,2020-09-08 02:05:20,284,3,175,4,3,...,0.679714,0.467279,0.672852,0.470161,0.598288,0.491629,0.679714,0.467279,0.672852,0.470161
1484,7404,6879,A030000111,-1,2020-10-13 09:49:18,673,5,144,4,5,...,0.701733,0.458110,0.647933,0.477910,0.598288,0.491629,0.701733,0.458110,0.647933,0.477910
1485,7416,9017,A050000193,-1,2020-10-04 02:44:41,878,6,184,3,6,...,0.658649,0.474735,0.770746,0.422156,0.663468,0.472688,0.658649,0.474735,0.770746,0.422156
1486,7417,9017,A050000193,-1,2020-09-06 13:09:15,878,6,184,3,6,...,0.658649,0.474735,0.770746,0.422156,0.663468,0.472688,0.658649,0.474735,0.770746,0.422156
